<a href="https://colab.research.google.com/github/mancinimassimiliano/DeepLearningLab/blob/master/Lab3/solution/finetune_alexnet_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Mount your Google drive folder on Colab

We are going to put data in google drive and then copy from google drive to colab local drive.

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

### Download dataset

After mounting the drive we will download OfficeHome dataset. 

Add [this](https://drive.google.com/file/d/10xMJi5uD8kVh9xkksq1pXngpUl-ckRBa/view?usp=sharing) .tar file to your Unitn google drive.



Copy the .tar file from gdrive to local drive. This will make data loading faster.

First create a directory with `!mkdir dataset` in your current path

In [0]:
!mkdir dataset

In [0]:
!cp "gdrive/My Drive/datasets/OfficeHomeDataset.tar" dataset/

In [0]:
!ls dataset

OfficeHomeDataset.tar


Unzip the .tar file

```
!tar -xf OfficeHomeDataset.tar
```
    Wait till its unzipped.

In [0]:
ls dataset

OfficeHomeDataset_10072016/  OfficeHomeDataset.tar


In [0]:
# unzip here
!tar -xf OfficeHomeDataset.tar

In [0]:
# import necessary libraries
import torch
import torchvision
import torch.nn.functional as F
import torchvision.transforms as T

# Library needed for visualization purposes
from tensorboardcolab import TensorBoardColab

# Instantiate visualizer
tb = TensorBoardColab(graph_path='./log')

Using TensorFlow backend.


Wait for 8 seconds...
TensorBoard link:
https://2663caa2.ngrok.io


### Alexnet 

![Alexnet architecture](https://www.oreilly.com/library/view/tensorflow-for-deep/9781491980446/assets/tfdl_0106.png =850x250)

This is the network architecture of [Alexnet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf). In this tutorial we are going to finetune a pre-trained Alexnet for classifying Office-Home images. Alexnet has been pre-trained on ILSVRC-2012 dataset, a dataset that has more than **1 million** images with 1000 classes.

***A little bit about the Office-Home dataset.***

Office Home has images from 4 different domains with each domain having 65 distinct categories. In this lab session we are going to use the `Real World` domain.

![Office-Home](http://hemanthdv.github.io/profile/images/DataCollage.jpg)

***Steps for Fine tuning Alexnet***

1.   Discard the final layer (or the output layer) of Alexnet which contains 1000 output neurons. 
2.   Randomly initialize the final layer with the number of output categories present in the dataset using `torch.nn.Linear`. In this case its 65 because OfficeHome has 65 classes. Keep all the other layers intact.
3. Train the network with a low learning rate for the pre-initialized layers and and a higher learning rate for the newly initialized layer. 



### How to load datasets from folders containing images

We have already seen that PyTorch's `torchvision` package provides some very basic dataloaders like `torchvision.datasets.MNIST`, `torchvision.datasets.SVHN`, etc. But it might happen (and is frequently the case) that we need to load our own datasets from folders.

PyTorch is kind enough to provide `torchvision.datasets.ImageFolder` for loading datasets from folders with just one line of code. But we need to ensure that the images in the folders are arranged in a certain way. A sample format is shown below:

OfficeHomeDataset

        |
        |--- Alarm_Clock
        |                
        |      |--- 00046.jpg
        |      |--- 00050.jpg
        |          
        |--- Couch
               | --- 00007.jpg
               | --- 00023.jpg
 
 In other words, the parent folder (*OfficeHomeDataset*) contains the sub-folders (e.g *Alarm_Clock*, *Couch*). These sub-folders are the *classes*. Further, each sub-folder contains a bunch of images (eg. 00046.jpg, 00050.jpg for the class *Alarm_Clock*). Internally, PyTorch will assign a class label to each sub-folder and will also load the corresponding images.
 
 More details goes [here](https://pytorch.org/docs/stable/torchvision/datasets.html#torchvision.datasets.ImageFolder).
 
N.B. Your own folder structure might look different, so provide path to your parent folder accordingly.

### Define the function that fetches a data loader that is then used during iterative training.

We are going to see some more PyTorch data transformations during loading the data.

In [0]:
'''
Input arguments:
  batch_size: mini batch size used during training
  img_root: path to the dataset parent folder. 
            The folder just above the sub-folders or class folders
'''

def get_data(batch_size, img_root):
  
  # Prepare data transformations for the train loader
  transform = list()
  transform.append(T.Resize((256, 256)))                      # Resize each PIL image to 256 x 256
  transform.append(T.RandomCrop((224, 224)))                  # Randonly crop a 224 x 224 patch
  transform.append(T.ToTensor())                              # converts Numpy to Pytorch Tensor
  transform.append(T.Normalize(mean=[0.485, 0.456, 0.406], 
                               std=[0.229, 0.224, 0.225]))    # Normalize with ImageNet mean
  transform = T.Compose(transform)                            # Composes the above transformations into one.
    
  # Load data
  officehome_dataset = torchvision.datasets.ImageFolder(root=img_root, transform=transform)
  
  # Create train and test splits
  # We will create a 80:20 % train:test split
  num_samples = len(officehome_dataset)
  training_samples = int(num_samples * 0.8 + 1)
  test_samples = num_samples - training_samples

  training_data, test_data = torch.utils.data.random_split(officehome_dataset, 
                                                           [training_samples, test_samples])

  # Initialize dataloaders
  train_loader = torch.utils.data.DataLoader(training_data, batch_size, shuffle=True)
  test_loader = torch.utils.data.DataLoader(test_data, batch_size, shuffle=False)
  
  return train_loader, test_loader

### Define Alexnet model

PyTorch provides a bunch of pre-trained models whose parameters have been learned on ImageNet dataset. All these models can be found [here](https://pytorch.org/docs/stable/torchvision/models.html).

We will load a pre-trained Alexnet using PyTorch's `torchvision` package. Then we will re-initialize the output layer suited for our classification task.

Before that lets have a look at the [code](https://github.com/pytorch/vision/blob/master/torchvision/models/alexnet.py) of Alexnet.

In [0]:
'''
Input arguments
  num_classes: number of classes in the dataset.
               This is equal to the number of output neurons.
'''

def initialize_alexnet(num_classes):
  # load the pre-trained Alexnet
  alexnet = torchvision.models.alexnet(pretrained=True)
  
  # get the number of neurons in the penultimate layer
  in_features = alexnet.classifier[6].in_features
  
  # re-initalize the output layer
  alexnet.classifier[6] = torch.nn.Linear(in_features=in_features, 
                                          out_features=num_classes)
  
  return alexnet

### Visualize our custom Alexnet model

In [0]:
print(initialize_alexnet(65))

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p

### Define cost function

In [0]:
def get_cost_function():
  cost_function = torch.nn.CrossEntropyLoss()
  return cost_function

### Define the optimizer

The optimizer will be different from the previous experiments.

Previously the *learning rate* of all the layers of the network was the same.

Now, we will have different layers learning at a different rate. The pre-trained layers need to be updated at a lesser rate than the newly initialized layer. Details are available [here](https://pytorch.org/docs/stable/optim.html).

In [0]:
def get_optimizer(model, lr, wd, momentum):
  
  # we will create two groups of weights, one for the newly initialized layer
  # and the other for rest of the layers of the network
  
  final_layer_weights = []
  rest_of_the_net_weights = []
  
  # we will iterate through the layers of the network
  for name, param in model.named_parameters():
    if name.startswith('classifier.6'):
      final_layer_weights.append(param)
    else:
      rest_of_the_net_weights.append(param)
  
  # so now we have divided the network weights into two groups.
  # We will train the final_layer_weights with learning_rate = lr
  # and rest_of_the_net_weights with learning_rate = lr / 10
  
  optimizer = torch.optim.SGD([
      {'params': rest_of_the_net_weights},
      {'params': final_layer_weights, 'lr': lr}
  ], lr= lr / 10, weight_decay = wd, momentum = momentum)
  
  return optimizer

### Train and test functions

In [0]:
def test(net, data_loader, cost_function, device='cuda:0'):
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.

  net.eval() # Strictly needed if network contains layers which has different behaviours between train and test
  with torch.no_grad():
    for batch_idx, (inputs, targets) in enumerate(data_loader):
      # Load data into GPU
      inputs = inputs.to(device)
      targets = targets.to(device)
        
      # Forward pass
      outputs = net(inputs)

      # Apply the loss
      loss = cost_function(outputs, targets)

      # Better print something
      samples+=inputs.shape[0]
      cumulative_loss += loss.item() # Note: the .item() is needed to extract scalars from tensors
      _, predicted = outputs.max(1)
      cumulative_accuracy += predicted.eq(targets).sum().item()

  return cumulative_loss/samples, cumulative_accuracy/samples*100


def train(net,data_loader,optimizer,cost_function, device='cuda:0'):
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.

  
  net.train() # Strictly needed if network contains layers which has different behaviours between train and test
  for batch_idx, (inputs, targets) in enumerate(data_loader):
    # Load data into GPU
    inputs = inputs.to(device)
    targets = targets.to(device)
      
    # Forward pass
    outputs = net(inputs)

    # Apply the loss
    loss = cost_function(outputs,targets)

    # Reset the optimizer
      
    # Backward pass
    loss.backward()
    
    # Update parameters
    optimizer.step()
    
    optimizer.zero_grad()

    # Better print something, no?
    samples+=inputs.shape[0]
    cumulative_loss += loss.item()
    _, predicted = outputs.max(1)
    cumulative_accuracy += predicted.eq(targets).sum().item()

  return cumulative_loss/samples, cumulative_accuracy/samples*100

### Wrapping everything up

Finally, we need a main function which initializes everything + the needed hyperparameters and loops over multiple epochs (printing the results).

In [0]:
'''
Input arguments
  batch_size: Size of a mini-batch
  device: GPU where you want to train your network
  weight_decay: Weight decay co-efficient for regularization of weights
  momentum: Momentum for SGD optimizer
  epochs: Number of epochs for training the network
  num_classes: Number of classes in your dataset
  visualization_name: Name of the visualization folder
  img_root: The root folder of images
'''

def main(batch_size=128, 
         device='cuda:0', 
         learning_rate=0.001, 
         weight_decay=0.000001, 
         momentum=0.9, 
         epochs=50, 
         num_classes=65, 
         visualization_name='alexnet_sgd', 
         img_root=None):
  
  train_loader, test_loader = get_data(batch_size=batch_size, 
                                       img_root=img_root)
  
  net = initialize_alexnet(num_classes=num_classes).to(device)
  
  optimizer = get_optimizer(net, learning_rate, weight_decay, momentum)
  
  cost_function = get_cost_function()

  print('Before training:')
  train_loss, train_accuracy = test(net, train_loader, cost_function)
  test_loss, test_accuracy = test(net, test_loader, cost_function)

  print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
  print('\t Test loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
  print('-----------------------------------------------------')
  
  # Add values to plots
  tb.save_value('Loss/train_loss', visualization_name, 0, train_loss)
  tb.save_value('Loss/test_loss', visualization_name, 0, test_loss)
  tb.save_value('Accuracy/train_accuracy', visualization_name, 0, train_accuracy)
  tb.save_value('Accuracy/test_accuracy', visualization_name, 0, test_accuracy)
    
  # Update plots 
  tb.flush_line(visualization_name)

  for e in range(epochs):
    train_loss, train_accuracy = train(net, train_loader, optimizer, cost_function)
    test_loss, test_accuracy = test(net, test_loader, cost_function)
    print('Epoch: {:d}'.format(e+1))
    print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
    print('\t Test loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
    print('-----------------------------------------------------')
    
    # Add values to plots
    tb.save_value('Loss/train_loss', visualization_name, e + 1, train_loss)
    tb.save_value('Loss/test_loss', visualization_name, e + 1, test_loss)
    tb.save_value('Accuracy/train_accuracy', visualization_name, se + 1, train_accuracy)
    tb.save_value('Accuracy/test_accuracy', visualization_name, e + 1, test_accuracy)
    
    # Update plots 
    tb.flush_line(visualization_name)

  print('After training:')
  train_loss, train_accuracy = test(net, train_loader, cost_function)
  test_loss, test_accuracy = test(net, test_loader, cost_function)

  print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
  print('\t Test loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
  print('-----------------------------------------------------')

Let's train!

In [0]:
main(visualization_name='alexnet_sgd_0.01_RW', 
     img_root = '/content/dataset/OfficeHomeDataset_10072016/Real World')

Before training:
	 Training loss 0.03591, Training accuracy 1.15
	 Test loss 0.03612, Test accuracy 1.26
-----------------------------------------------------
Epoch: 1
	 Training loss 0.02808, Training accuracy 19.36
	 Test loss 0.01872, Test accuracy 45.35
-----------------------------------------------------
Epoch: 2
	 Training loss 0.01381, Training accuracy 57.40
	 Test loss 0.01213, Test accuracy 60.51
-----------------------------------------------------
Epoch: 3
	 Training loss 0.00930, Training accuracy 68.10
	 Test loss 0.01024, Test accuracy 64.41
-----------------------------------------------------
Epoch: 4
	 Training loss 0.00737, Training accuracy 74.44
	 Test loss 0.00965, Test accuracy 66.59
-----------------------------------------------------
Epoch: 5
	 Training loss 0.00593, Training accuracy 79.92
	 Test loss 0.00932, Test accuracy 67.39
-----------------------------------------------------
Epoch: 6
	 Training loss 0.00490, Training accuracy 82.33
	 Test loss 0.0091

In [0]:
rm -rf log